# ２０２２年の車上荒らしについて分析してみた。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=[19.2,10.8]
plt.rcParams['font.family']= 'Hiragino Maru Gothic Pro'

## 1. データクレンジング

In [ ]:
#CSV読み込み
df1=pd.read_csv("tokyo_2022syazyounerai.csv")
df1.head()

In [ ]:
#カラム削除(市区町村コード,町丁目)
df1=df1.drop(["市区町村コード（発生地）","町丁目（発生地）"],axis=1)
#年月日をdatetime型に変換
df1["発生年月日（始期）"]=pd.to_datetime(df1["発生年月日（始期）"])
df1.head()

In [ ]:
df1.info()

In [ ]:
#欠損値確認
print(df1.isnull().sum())
#欠損値抽出
dfnull=df1[df1.isnull().any(axis=1)]
display(dfnull)

In [ ]:
#欠損値削除
df1=df1.dropna()
#発生時「不明」を削除して、int型に変換
df1=df1.drop(index=df1[df1["発生時（始期）"]=="不明"].index)
df1["発生時（始期）"]=df1["発生時（始期）"].astype(int)

In [ ]:
#2022年に発生した事件のみ抽出
df1["発生年"]=df1["発生年月日（始期）"].dt.strftime("%Y")
df1=df1[df1["発生年"]=="2022"]
df1=df1.drop(columns="発生年")
df1["発生年月"]=df1["発生年月日（始期）"].dt.strftime("%Y%m")
df1.info()

## 2.統計分析

In [ ]:
#東京都内の各市区町村の犯罪発生件数
df_bar=df1.groupby("市区町村（発生地）").count()[["都道府県（発生地）"]]
df_bar=df_bar.sort_values("都道府県（発生地）",ascending=False)
df_bar.head()

In [ ]:
#平均・分散・標準偏差
print("都内の年間発生件数")
print("平均",df_bar["都道府県（発生地）"].mean())
print("分散",df_bar["都道府県（発生地）"].var())
print("標準偏差",df_bar["都道府県（発生地）"].std())

In [ ]:
#ポアソン分布
from scipy.stats import skew,poisson,kurtosis

bar_mean=df_bar.mean()
x=np.arange(0,60,1)
#確率質量関数
poi=poisson.pmf(x,bar_mean)
#累積関数
cdf=poisson.cdf(x,bar_mean)
#尖度
sk=skew(poi)
#歪度
kur=kurtosis(poi)

In [ ]:
%matplotlib inline

plt.bar(x,poi)
plt.plot(x, poi)
plt.title("ポアソン分布",fontsize=30)
plt.xlabel("件数",fontsize=18)
plt.ylabel("確率",fontsize=18)
plt.grid()
plt.show()

print('歪度: {0:.2f}'.format(kur))
print('尖度: {0:.2f}'.format(sk))

#累積分布関数
%matplotlib inline

plt.title("累積分布",fontsize=30)
plt.step(x,cdf)
plt.xlabel("件数",fontsize=18)
plt.ylabel("累積確率",fontsize=18)
plt.grid()
plt.show()

## 自治体別統計データ

In [ ]:
#グラフにプロット
df_bar.plot.bar()
plt.hlines(y=df_bar["都道府県（発生地）"].mean(),xmin=0,xmax=51,color="black")
plt.title("自治体別発生件数",fontsize=30)
plt.xlabel("市区町村",fontsize=18)
plt.ylabel("発生件数",fontsize=18)
plt.show()

In [ ]:
#施錠の有無
df1.groupby(["施錠関係"]).count()[["発生年月日（始期）"]]

In [ ]:
lock=df1[df1["施錠関係"]=="施錠した"]
lock=lock.groupby("市区町村（発生地）").count()[["施錠関係"]]
lock=lock.sort_values("施錠関係",ascending=False)

lock.plot.bar()
plt.title("施錠した",fontsize=30)
plt.xlabel("市区町村",fontsize=18)
plt.ylabel("件数",fontsize=18)
plt.show()

no_lock=df1[df1["施錠関係"]=="施錠せず"]
no_lock=no_lock.groupby("市区町村（発生地）").count()[["施錠関係"]]
no_lock=no_lock.sort_values("施錠関係",ascending=False)

no_lock.plot.bar()
plt.title("施錠してない",fontsize=30)
plt.xlabel("市区町村",fontsize=18)
plt.ylabel("件数",fontsize=18)
plt.show()

In [ ]:
#現金被害の有無
no_money=df1[df1["現金被害の有無"]=="あり"]
no_money=no_money.groupby("市区町村（発生地）").count()[["現金被害の有無"]]
no_money=no_money.sort_values("現金被害の有無",ascending=False)


no_money.plot.bar()
plt.title("現金被害　あり",fontsize=30)
plt.xlabel("市区町村",fontsize=18)
plt.ylabel("件数",fontsize=18)
plt.show()

money=df1[df1["現金被害の有無"]=="なし"]
money=no_money.groupby("市区町村（発生地）").count()[["現金被害の有無"]]
money=no_money.sort_values("現金被害の有無",ascending=False)

money.plot.bar()
plt.title("現金被害なし",fontsize=30)
plt.xlabel("市区町村",fontsize=18)
plt.ylabel("件数",fontsize=18)
plt.show()

In [ ]:
df1.groupby(["発生年月","施錠関係","現金被害の有無"]).count()[["市区町村（発生地）"]]

In [ ]:
#施錠したかつ現金被害ありのデータ抽出
brutal=df1[df1["現金被害の有無"]=="あり"]
brutal=brutal[brutal["施錠関係"]=="施錠した"]
brutal=brutal.groupby(["市区町村（発生地）"]).count()[["発生年月日（始期）"]]
brutal=brutal.sort_values("発生年月日（始期）",ascending=False)
display(brutal.head())
display("合計",brutal.sum())
display("平均",brutal.mean())

In [ ]:
#ポアソン分布
from scipy.stats import poisson
brutal_mean=brutal.mean()
x=np.arange(0,20,1)
#確率質量関数
b_poi=poisson.pmf(x,brutal_mean)
#累積関数
cdf=poisson.cdf(x,brutal_mean)

plt.bar(x,b_poi)
plt.plot(x, b_poi,color="red")
plt.title("ポアソン分布",fontsize=30)
plt.xlabel("件数",fontsize=18)
plt.ylabel("確率",fontsize=18)
plt.grid()
plt.show()

In [ ]:
#月単位で件数を抽出
df1["発生年月"] = df1["発生年月日（始期）"].dt.strftime("%Y%m")
df_month=df1.groupby(["発生年月"]).count()[["市区町村（発生地）"]]
df_month.head()

In [ ]:
#コレログラム
%matplotlib inline

plt.xlabel("発生年月",fontsize=18)
plt.ylabel("発生件数",fontsize=18)
plt.stem(np.sort(df1["発生年月"].unique()),df_month)

In [ ]:
#ロジスティック回帰(施錠関係の有無)
from sklearn.linear_model import LogisticRegression

logistic_x=np.array([df1["発生時（始期）"]]).reshape(-1, 1)
logistic_ylock=df1["施錠関係"].map({"施錠した":0,"施錠せず":1})
logistic_regression=LogisticRegression()
logistic_lock=logistic_regression.fit(logistic_x,logistic_ylock)

In [ ]:
#性能評価
from sklearn.metrics import log_loss

lock_proba=logistic_lock.predict_proba(logistic_x)
score=log_loss(logistic_ylock,lock_proba)
print(lock_proba)
print(f'logloss:{score:.4f}')

In [ ]:
#混同行列
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report

lock_pred=logistic_lock.predict(logistic_x)
lock_cm=confusion_matrix(logistic_ylock,lock_pred)
sns.heatmap(lock_cm,annot=True,cmap='Oranges',fmt='d')
print(classification_report(logistic_ylock,lock_pred))

In [ ]:
df1.groupby(["現金被害の有無"]).count()[["発生年月日（始期）"]]

In [ ]:
#ロジスティック回帰(現金被害のの有無)
logistic_ymoney=df1["現金被害の有無"].map({"あり":0,"なし":1})
logistic_money=logistic_regression.fit(logistic_x,logistic_ymoney)

In [ ]:
#性能評価
from sklearn.metrics import log_loss

money_proba=logistic_money.predict_proba(logistic_x)
score=log_loss(logistic_ymoney,money_proba)
print(money_proba)
print(f'logloss:{score:.4f}')

In [ ]:
#混同行列
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report

money_pred=logistic_money.predict(logistic_x)
money_cm=confusion_matrix(logistic_ymoney,money_pred)
sns.heatmap(money_cm,annot=True,cmap='Oranges',fmt='d')
print(classification_report(logistic_ymoney,money_pred))

## 3. 人口と犯罪発生件数についての予測モデル

In [ ]:
population=pd.read_csv("jy22qv0100.csv")
population.head()

In [ ]:
population.isnull().sum()
population_null=population[population.isnull().any(axis=1)]
population_null

In [ ]:
#欠損値削除
population=population.dropna()
population=population.drop(population.index[[0,1]])
#空白削除
population["地域"]=population["地域"].str.strip()
population.info()

In [ ]:
population_num=population[["地域","令和4年1月1日現在／人口／総数(人)",]]
population_num.head()

In [ ]:
#データ統合
df_new=df_bar.reset_index()
df_new=df_new.rename(columns={'市区町村（発生地）': '地域'})
df_pop=pd.merge(df_new,population_num,on='地域',how='left')
df_pop.head()

In [ ]:
#欠損値確認
df_pop.isnull().sum()

In [ ]:
#正規化
from sklearn.preprocessing import MinMaxScaler

mm=MinMaxScaler()
df_mm=mm.fit_transform(df_pop[["都道府県（発生地）","令和4年1月1日現在／人口／総数(人)"]])

#標準化
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
df_sc=sc.fit_transform(df_mm)
df_sc=pd.DataFrame(df_mm)
df_sc.head()

In [ ]:
#散布図
%matplotlib inline
plt.scatter(df_sc[1],df_sc[0])
plt.xlabel("発生件数",fontsize=18)
plt.ylabel("人口",fontsize=18)
plt.show()

In [ ]:
#線形単回帰
from sklearn.linear_model import LinearRegression

linear_regression=LinearRegression()
x=np.array(df_sc[0]).reshape(-1, 1)
y=np.array(df_sc[1]).reshape(-1, 1)
linear_reg=linear_regression.fit(x,y)
print("回帰係数：",linear_reg.coef_,"切片：",linear_reg.intercept_)
linear_pred=linear_reg.predict(x)
print("決定係数：",linear_regression.score(x, y))

In [ ]:
#多項式回帰
from sklearn.preprocessing import PolynomialFeatures

#バイアス削除
quadratic=PolynomialFeatures(degree=2,include_bias=False)
x_quad=quadratic.fit_transform(x)
poly_reg=linear_regression.fit(x_quad,y)
print("回帰係数：",poly_reg.coef_,"切片：",poly_reg.intercept_)
poly_pred=poly_reg.predict(x_quad)
print("決定係数：",linear_regression.score(x_quad, y))

In [ ]:
#リッジ回帰
from sklearn.linear_model import Ridge

ridge_regression=Ridge(alpha=0.1)
ridge_reg=ridge_regression.fit(x_quad,y)
print("回帰係数：",ridge_reg.coef_,"切片：",ridge_reg.intercept_)
ridge_pred=ridge_reg.predict(x_quad)
print("決定係数：",ridge_regression.score(x_quad, y))

In [ ]:
#線形サポートベクター回帰
from sklearn.svm import SVR

svr=SVR(kernel='linear')
svr_reg=svr.fit(x,y.ravel())
print("回帰係数：",svr_reg.coef_,"切片：",svr_reg.intercept_)
svr_pred=svr_reg.predict(x)
print("決定係数：",svr.score(x, y))

In [ ]:
#多項式サポートベクター回帰

svr_poly=SVR(kernel='poly')
svr_polyreg=svr_poly.fit(x,y.ravel())
svr_poly_pred=svr_polyreg.predict(x)
print("決定係数：",svr_poly.score(x, y))

In [ ]:
#ガウスカーネルサポートベクター回帰

svr_rbf=SVR(kernel='rbf')
svr_rbfreg=svr_rbf.fit(x,y.ravel())
svr_rbf_pred=svr_rbfreg.predict(x)
print("決定係数：",svr_rbf.score(x, y))

In [ ]:
#決定木回帰
from sklearn.tree import DecisionTreeRegressor

tree_regression=DecisionTreeRegressor(max_depth=3)
tree_reg=tree_regression.fit(x, y)
tree_pred=tree_reg.predict(x)
print("決定係数：",tree_regression.score(x, y))

In [ ]:
#ランダムフォレスト回帰
from sklearn.ensemble import RandomForestRegressor

rf_regression=RandomForestRegressor()
rf_reg=rf_regression.fit(x,y.ravel())
rf_pred=rf_reg.predict(x)
print("決定係数：",rf_regression.score(x, y))

In [ ]:
#ポアソン回帰
from sklearn.linear_model import PoissonRegressor

poisson_regression=PoissonRegressor()
poisson_reg=poisson_regression.fit(x,y.ravel())
print("回帰係数：",poisson_reg.coef_,"切片：",poisson_reg.intercept_)
poisson_pred=poisson_reg.predict(x)

In [ ]:
%matplotlib inline
plt.scatter(x,y)
plt.xlabel("発生件数",fontsize=18)
plt.ylabel("人口",fontsize=18)
plt.plot(x,linear_pred,c="r",label="線形")
plt.plot(x,poly_pred,c="y",label="多項式")
plt.plot(x,ridge_pred,c="m",label="リッジ")
plt.plot(x,svr_pred,c="g",label="線形サポートベクター")
plt.plot(x,svr_poly_pred,label="多項式サポートベクター")
plt.plot(x,svr_rbf_pred,c="c",label="ガウスカーネルサポートベクター")
plt.plot(x,tree_pred,label="決定木")
plt.plot(x,rf_pred,label="ランダムフォレスト")
plt.plot(x,poisson_pred,c="b",label="ポアソン")
plt.xlabel("人口",fontsize=18)
plt.ylabel("発生件数",fontsize=18)
plt.grid()
plt.legend()
plt.show()

In [ ]:
#各モデルの性能評価
from sklearn.metrics import mean_squared_error
#MSE(平均二乗誤差)
print('線形回帰　MSE: % 4f' %mean_squared_error(y,linear_pred))
print('多項式回帰　MSE: % 4f' %mean_squared_error(y,poly_pred))
#RMSE(平均二乗平方根誤差)
print('線形回帰　RMSE: % 4f' %(np.sqrt(mean_squared_error(y,linear_pred))))
print('多項式回帰　RMSE: % 4f' %(np.sqrt(mean_squared_error(y,poly_pred))))

In [ ]:
#残差平方和プロット
linear_e=(y-linear_pred)**2
poly_e=(y-poly_pred)**2
ridge_e=(y-ridge_pred)**2
plt.plot(np.sort((linear_e).ravel()),label="線形回帰")
plt.plot(np.sort((poly_e).ravel()),label="多項式回帰")
plt.plot(np.sort((ridge_e).ravel()),label="リッジ回帰")
plt.hlines(y=np.sqrt(mean_squared_error(y,linear_pred)),xmin=0,xmax=51,color="blue")
plt.hlines(y=np.sqrt(mean_squared_error(y,poly_pred)),xmin=0,xmax=51,color="orange")
plt.hlines(y=np.sqrt(mean_squared_error(y,ridge_pred)),xmin=0,xmax=51,color="green")
plt.title("残差平方和",fontsize=32)
plt.grid()
plt.legend()
plt.show()

In [ ]:
#残差プロット(線形回帰、多項式、リッジ)
plt.scatter(linear_pred,y-linear_pred,label="線形")
plt.scatter(poly_pred,y-poly_pred,label="多項式")
plt.scatter(ridge_pred,y-ridge_pred,label="リッジ")
plt.title("残差プロット",fontsize=32)
plt.xlabel("予測値",fontsize=18)
plt.ylabel("残差",fontsize=18)
plt.hlines(y=0,xmin=-0.5,xmax=1.5,color="black")
plt.grid()
plt.legend()
plt.show()